### EXPLORING A DATASET
Before we can conduct data analysis, we must outline our goals and understand the information available through 3 different steps.
* Describing the variables in the heart disease data set.
* Loading the heart disease data.

### Method
Our objective is to conduct a multiclass classification with more than two categories to answer a predictive question: can we use the attributes of the patient available to us to predict and distinguish the presence of heart disease in the patient? Our Class will be num, the diagnosis of heart disease (angiographic disease status) ranging from 0 to 4:
* presence in severity (values 1,2,3,4) and,
* absence (value 0).  

Given the following attributes and its description:
1. age: age in years
2. sex: sex (1 = male; 0 = female)
3. cp: chest pain type
   * Value 1: typical angina
   * Value 2: atypical angina
   * Value 3: non-anginal pain
   * Value 4: asymptomatic
4. trestbps: resting blood pressure (in mm Hg on admission to the hospital)
5. chol: serum cholestoral in mg/dl
6. fbs: (fasting blood sugar > 120 mg/dl)  (1 = true; 0 = false)
7. restecg: resting electrocardiographic results
    * Value 0: normal
    * Value 1: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV)
    * Value 2: showing probable or definite left ventricular hypertrophy by Estes' criteria
8. thalach: maximum heart rate achieved
9. exang: exercise induced angina (1 = yes; 0 = no)
10. oldpeak: ST depression induced by exercise relative to rest
11. slope: the slope of the peak exercise ST segment
    * Value 1: upsloping
    * Value 2: flat
    * Value 3: downsloping
13. ca: number of major vessels (0-3) colored by flourosopy
14. thal: 3 = normal; 6 = fixed defect; 7 = reversable defect
15. num: diagnosis of heart disease (angiographic disease status)
    * Value 0: < 50% diameter narrowing
    * Value 1: > 50% diameter narrowing

We plan to conduct data analysis to build a classifier using three different predictors (variables/columns):
* age: age in years.* chol: serum cholestoral in mg/d.
* thalach: maximum heart rate achieved.


Each variable has been: is this a useful variable for prediction? Describe at least one way that you will visualize the results.

In [1]:
#loading in the library necessary
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.5     ✔ rsample      1.2.0
✔ dials        1.2.0     ✔ tune         1.1.2
✔ infer        1.0.5     ✔ workflows    1.1.3
✔ modeldata    1.2.0     ✔ workflowsets 1.0.1
✔ parsnip      1.1.1     ✔ yardstick    1.2.0
✔ recipes      1.0.8     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filt

In [2]:
# read data
processed_cleveland <- read_csv("data/processed.cleveland.data", col_names = F)

Rows: 303 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (2): X12, X13
dbl (12): X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X14

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [3]:
# renaming variables for readability 
colnames(processed_cleveland) <- c("age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", 
                                   "thalach", "exang", "oldpeak", "slope", "ca", "thal", "num")

processed_cleveland

age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>
63,1,1,145,233,1,2,150,0,2.3,3,0.0,6.0,0
67,1,4,160,286,0,2,108,1,1.5,2,3.0,3.0,2
67,1,4,120,229,0,2,129,1,2.6,2,2.0,7.0,1
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
57,1,4,130,131,0,0,115,1,1.2,2,1.0,7.0,3
57,0,2,130,236,0,2,174,0,0.0,2,1.0,3.0,1
38,1,3,138,175,0,0,173,0,0.0,1,?,3.0,0


In [4]:
#create the TRAIN SET and TEST SET
set.seed(2000)

cleveland_filter <- processed_cleveland |>
            select(age, chol, thalach, num) |>
            mutate(num = as_factor(num)) |>
            sample_n(300)
cleveland_filter

cleveland_split <- initial_split(cleveland_filter, prop = 0.75, strata = num)
cleveland_train <- training(cleveland_split) 
cleveland_test <- testing(cleveland_split)

age,chol,thalach,num
<dbl>,<dbl>,<dbl>,<fct>
52,325,172,0
56,184,105,1
40,167,114,3
⋮,⋮,⋮,⋮
64,263,105,0
62,231,146,0
49,269,163,0


### Preliminary Exploratory Data Analysis

In [6]:
#find the number and percentage of differing presense of heart disease observations in our data set

num_obs <- nrow(cleveland_train)
    cleveland_train |>
    group_by(num) |>
    summarize(
        count = n(),
        percentage = n() / num_obs * 100
    )

num,count,percentage
<fct>,<int>,<dbl>
0,123,55.156951
1,39,17.488789
2,27,12.107623
3,26,11.659193
4,8,3.587444


In [ ]:
#draw a scatter plot to visualize the relationship between the chose age and chol (serum cholestoral in mg/d) predictors/variables


In [ ]:
#draw a scatter plot to visualize the relationship between the chose age and thalach (maximum heart rate achieved) predictors/variables


In [ ]:
#draw a scatter plot to visualize the relationship between the chose chol (serum cholestoral in mg/d) and thalach (maximum heart rate achieved) predictors/variables
